<a href="https://colab.research.google.com/github/AreebAhmad-02/Nuvexa-Photos/blob/main/langchain_groq_unstructured_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build Your Own Retrieval Augmented Generation (RAG) Bot



## Tech used
- LangChain [link](https://www.langchain.com/)
- Unstructured [link](https://unstructured.io/)
- LangSmith [link](https://smith.langchain.com/)
- Qdrant Cloud [link](https://cloud.qdrant.io)
- Groq API [link](https://console.groq.com/playground)
- Llama3 via Groq API
- Fastembed [link](https://github.com/qdrant/fastembed)

## File type used
- PDF
- Markdown

### Some important links
- https://unstructured.io/
- https://unstructured-io.github.io/unstructured/index.html
- https://docs.unstructured.io/api-reference/api-services/python-sdk
- https://www.deeplearning.ai/short-courses/preprocessing-unstructured-data-for-llm-applications/

## Setup

In [1]:
#%%capture
!pip install "unstructured[all-docs]" unstructured-client watermark langchain-groq langchain fastembed qdrant_client python-dotenv

In [2]:
# Install Requirements
!apt-get -qq install poppler-utils tesseract-ocr
# Upgrade Pillow to latest version
%pip install -q --user --upgrade pillow

Selecting previously unselected package poppler-utils.
(Reading database ... 121913 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.4_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.4) ...
Selecting previously unselected package tesseract-ocr-eng.
Preparing to unpack .../tesseract-ocr-eng_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr-osd.
Preparing to unpack .../tesseract-ocr-osd_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr.
Preparing to unpack .../tesseract-ocr_4.1.1-2.1build1_amd64.deb ...
Unpacking tesseract-ocr (4.1.1-2.1build1) ...
Setting up tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Setting up poppler-utils (22.02.0-2ubuntu0.4) ...
Setting up tess

In [3]:
%load_ext watermark

In [4]:
%watermark

Last updated: 2024-06-11T11:55:06.491555+00:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.85+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [6]:
from unstructured.chunking.title import chunk_by_title
from unstructured.partition.md import partition_md
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import dict_to_elements

In [7]:
#import langchain, groq, fastembed, qdrant_client, unstructured

In [8]:
%watermark --iversions

In [10]:
import unstructured.partition

help(unstructured.partition)

Help on package unstructured.partition in unstructured:

NAME
    unstructured.partition

PACKAGE CONTENTS
    api
    auto
    common
    csv
    doc
    docx
    email
    epub
    html
    image
    json
    lang
    md
    model_init
    msg
    odt
    org
    pdf
    pdf_image (package)
    ppt
    pptx
    rst
    rtf
    strategies
    text
    text_type
    tsv
    utils (package)
    xlsx
    xml

FILE
    /usr/local/lib/python3.10/dist-packages/unstructured/partition/__init__.py




## Preprocess the PDF

In [11]:
partition_pdf??

In [12]:
import os.path
from google.colab import files
# Specify the path to your file
filename = "/content/data/The_Alchemist.pdf"




# Check if the file exists
if os.path.exists(filename):
    print("The file exists.")
else:
    print("The file does not exist.")

The file does not exist.


In [13]:
!pip install poppler-utils


In [18]:
from unstructured.partition.pdf import partition_pdf

# Specify the path to your PDF file
filename = "/content/data/The_Alchemist.pdf"

#path = "images"

# Extract images, tables, and chunk text
pdf_elements = partition_pdf(
    filename=filename,

    strategy = "hi_res",
    # hi_res_model_name="yolox",

    infer_table_structure=True,
    chunking_strategy="by_title",
    multipage_sections = True,
    # max_characters=3000,
    new_after_n_chars=3000,
    # combine_text_under_n_chars=100,
    #extract_image_block_output_dir=path,
)

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [57]:
# Function to convert metadata to a serializable format
def convert_metadata(metadata):
    if isinstance(metadata, dict):
        return {k: convert_metadata(v) for k, v in metadata.items()}
    elif isinstance(metadata, list):
        return [convert_metadata(v) for v in metadata]
    elif isinstance(metadata, (str, int, float, bool, type(None))):
        return metadata
    else:
        return str(metadata)  # Convert any other types to string

In [59]:
copy_of_pdf_elements  = pdf_elements.copy()

In [60]:
import json

elements_dicts = []
for el in copy_of_pdf_elements:
    elements_dicts.append({
        'category': el.category,
        'text': el.text,
        'metadata': convert_metadata(el.metadata),  # Convert metadata
    })

# Define the output JSON file path
output_filename = "pdf_elements.json"

# Save the elements to a JSON file
with open(output_filename, 'w', encoding='utf-8') as json_file:
    json.dump(elements_dicts, json_file, ensure_ascii=False, indent=4)

print(f"PDF elements have been saved to {output_filename}")

PDF elements have been saved to pdf_elements.json


In [19]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 611}

In [20]:
element_dict = [el.to_dict() for el in pdf_elements]

unique_types = set()

for item in element_dict:
    unique_types.add(item['type'])

print(unique_types)

{'CompositeElement'}


In [ ]:
# Extract images, tables, and chunk text
pdf_elements = partition_pdf(
    filename=filename,
    extract_images_in_pdf=False,
    strategy = "hi_res",
    hi_res_model_name="yolox",
    infer_table_structure=True,
    #chunking_strategy="by_title",
    max_characters=3000,
    #new_after_n_chars=3800,
    combine_text_under_n_chars=200,
    #extract_image_block_output_dir=path,
)

for element in pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
print(category_counts)

element_dict = [el.to_dict() for el in pdf_elements]

unique_types = set()

for item in element_dict:
    unique_types.add(item['type'])

print(unique_types)

In [40]:
pdf_elements[0].to_dict()

{'type': 'CompositeElement',
 'element_id': '85dc15464cf08778aa81b1199ac7121e',
 'text': 'ALCHEMIST\n\nArse\n\nPAULO COFLIO,\n\nTHE ALCHEMIST\n\nPAULO COELHO\n\nTRANSLATED BY ALAN R. CLARKE\n\nContents\n\nINTRODUCTION\n\nI remember receiving a letter from the American publisher Harper Collins…\n\nPROLOGUE\n\nThe alchemist picked up a book that someone\n\nin the…\n\nONE\n\nThe boy’s name was Santiago. Dusk was falling as the…\n\nTWO\n\nThe boy had been working for the crystal merchant for…\n\nEPILOGUE',
 'metadata': {'filetype': 'application/pdf',
  'languages': ['eng'],
  'last_modified': '2024-06-11T11:55:23',
  'page_number': 1,
  'orig_elements': 'eJzNWO9P40YQ/VesfGat3dnf941ytKByBEFOVUVPaHe9BovEjhJzd+jU/72ztiNxR9qSSE77hTDPHtme5zfzxrffJnEeF7Fu76pi8i6b0EhlSRWQUhhFhAqWGKoYMSIYQ5kKqmCTo2yyiK0rXOsw59skNM2qqGrXxnUXz91z89TePcTq/qFFBDi3mDPAX6qifUCUKSkQXTZV3aa821tDc3qUCY0/n46yIVQwhKw7sAXoExCZrJ/XbVyk57iqvsb5zdKFOPkTD5TVPLbPy9gduvow6e6mvn9y990t305ifT/51KHr9m7RFFVZxa4gQEEQqghjM8beSfkOeMpeYuZd/

In [41]:
pdf_elements[5].text

'The only honest response is: I don’t know. All I know is that, like Santiago the shepherd boy, we all need to be aware of our personal calling. What is a personal calling? It is God’s blessing, it is the path that God chose for you here on Earth. Whenever we do something that fills us with enthusiasm, we are following our legend. However, we don’t all have the courage to confront our own dream.\n\nWhy?'

In [23]:
tables = [el for el in pdf_elements if el.category == "Table"]
tables

[]

In [ ]:
table_html = tables[0].metadata.text_as_html

In [ ]:
from io import StringIO
from lxml import etree

parser = etree.XMLParser(remove_blank_text=True)
file_obj = StringIO(table_html)
tree = etree.parse(file_obj, parser)
print(etree.tostring(tree, pretty_print=True).decode())

In [34]:
categoriees = [el.category for el in pdf_elements]

In [36]:
set(categoriees)

{'CompositeElement'}

In [38]:
# Find the element with text "References" and category "Title"
reference_title = [
    el for el in pdf_elements
    if el.text == "References"
    and el.category == "CompositeElements"
][0]

In [53]:
# searching for part two in every chunk
search_text = "part two"

def finding_text_in_chunks(search_text):
  # Iterate through the elements and search for the text
  found_elements = []
  for el in pdf_elements:
      if search_text.lower() in el.text.lower():
          found_elements.append(el)

  # Print out the elements that contain the search text
  if found_elements:
      print(f'Found "{search_text}" in the following elements:')
      for el in found_elements:
          print(f'Category: {el.category}, Text: {el.text[:200]}')
  else:
      print(f'"{search_text}" not found in any elements.')

  return found_elements

In [54]:
found_elements_for_about_publishers = finding_text_in_chunks("about the publisher")

Found "about the publisher" in the following elements:
Category: CompositeElement, Text: The boy reached the small, abandoned church just as night…

ABOUT THE AUTHOR

INTERNATIONAL ACCLAIM

BOOKS BY PAULO COELHO

CREDITS

COVER  

COPYRIGHT

ABOUT THE PUBLISHER
Category: CompositeElement, Text: PerfectBound™ and the PerfectBound™ logo are trademarks of HarperCollins Publishers, Inc.

Mobipocket Reader July 2005 ISBN 0-06-088268-9

Library of Congress Cataloging-in-Publication Data has been




In [72]:
found_elements_for_about_publishers[0].text

'The boy reached the small, abandoned church just as night…\n\nABOUT THE AUTHOR\n\nINTERNATIONAL ACCLAIM\n\nBOOKS BY PAULO COELHO\n\nCREDITS\n\nCOVER  \n\nCOPYRIGHT\n\nABOUT THE PUBLISHER'

In [70]:
# found_elements_for_about_publishers[0].text
found_elements_for_about_publishers[1].id

'1cf7f4d5e57b52cf10a7679d32eab0ae'

In [75]:
publisher_el_id = [el.id for el in found_elements_for_about_publishers]
print(publisher_el_id)

['57fd0772bc067337c6aceb324e55f476', '1cf7f4d5e57b52cf10a7679d32eab0ae']


In [76]:
len(pdf_elements)

611

In [80]:
# Remove elements containing the specific text from pdf_elements
pdf_elements = [el for el in pdf_elements if el.id not in publisher_el_id]


In [81]:
len(pdf_elements)

609

In [82]:
# checking agin for the about the publisher page in pdf_elements
finding_text_in_chunks("about the publisher")

"about the publisher" not found in any elements.


[]

### Filter out headers

In [61]:
headers = [el for el in pdf_elements if el.category == "Header"]

In [62]:
len(headers)

0

In [84]:
# Filters out elements from the `pdf_elements` list that have the category "Header".
h_pdf_elements = [el for el in pdf_elements if el.category != "Header"]

In [85]:
len(pdf_elements)

609

In [86]:
# lets again see some random index
pdf_elements[33].to_dict()

{'type': 'CompositeElement',
 'element_id': 'b033aa3f51cb1941d09871c276c08c31',
 'text': 'The only things that concerned the sheep were food and water. As long as the boy knew how to find the best pastures in Andalusia, they would be his friends. Yes, their days were all the same, with the seemingly endless hours between sunrise and dusk; and they had never read a book in their young lives, and didn’t understand when the boy told them about the sights of the cities. They were content with just food and water, and, in exchange, they generously gave of their wool, their company,',
 'metadata': {'filetype': 'application/pdf',
  'languages': ['eng'],
  'last_modified': '2024-06-11T11:55:23',
  'page_number': 18,
  'orig_elements': 'eJxtU9GOmzAQ/JUVzwkKkJBwfboPaHVS81Klp2jBC7hnbGSbEHS6f+/a5CpVuifYsWd3Zlgu7wkpGkj7qxTJEyRHKotjU9VbUdJxu6dCbLFG3BbHluq8yrNsd0w2kAzkUaBH5rwnjTFWSI2eXKwVLmby155k13tG8qKomPOAZyl8z2hWHvaMjkZqH3iXS1Yd0nID5bFIT68b+KyzrDw9gH2Zp4evkJXDSOIW52kIXl7kndTPERtKPvhAkKfGS6OvjULnrq

#### Let's still do some more exploration

In [ ]:
len(pdf_elements), len(md_elements)

(72, 1506)

In [ ]:
elements = chunk_by_title(pdf_elements + md_elements) # you can play around with the chunk_by_title arguments

In [ ]:
len(elements)

731

In [ ]:
pdf_elements[0].to_dict()

In [ ]:
pdf_elements[1].to_dict()

In [ ]:
elements[0].to_dict()

{'type': 'CompositeElement',
 'element_id': 'de7552d5-35e4-4f67-9e92-8bb73a59f958',
 'text': '3 2 0 2\n\n1 v 1 3 9 4 0 . 1 1 3 2 : v i X r a\n\nGPT4All: An Ecosystem of Open Source Compressed Language Models\n\nYuvanesh Anand Nomic AI yuvanesh@nomic.ai\n\nZach Nussbaum Nomic AI zach@nomic.ai\n\nAdam Treat Nomic AI adam@nomic.ai\n\nAaron Miller Nomic AI aaron@nomic.ai\n\nRichard Guo Nomic AI richard@nomic.ai\n\nBen Schmidt Nomic AI ben@nomic.ai\n\nGPT4All Community Planet Earth\n\nBrandon Duderstadt∗ Nomic AI brandon@nomic.ai\n\nAndriy Mulyar∗ Nomic AI andriy@nomic.ai',
 'metadata': {'file_directory': 'data',
  'filename': 'gpt4all.pdf',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'last_modified': '2024-05-03T13:31:00',
  'page_number': 1,
  'orig_elements': 'eJzlWNtuGzcQ/RVCz86W94uf6rRBUKBJg8YF2rqBwMvQWmAvwmqVRgny75292FZiubEL6MH1k3QOh1guz+FwZi8+LaCCGpp+WabFKVkEGlXMMTvhOTPKMeAmqeiMlSorFxYnZFFD75PvPcZ/WsS27VLZ+B42I678rt32yxWUl6seGS6Ewzkz/XeZ+hWyTCuJ7Lotm36Yd3EhVYGMEbyw707I

In [87]:
chunk_by_title??

In [ ]:
#chunk_elements = chunk_by_title((pdf_elements + md_elements),combine_text_under_n_chars=100,max_characters=3000)
#len(chunk_elements)

## Load the Documents into the Vector DB

In [89]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.7 MB/s eta 0:00:00


In [90]:
import os
from langchain_core.documents import Document
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Qdrant

In [ ]:
documents = []
for element in elements:
    metadata = element.metadata.to_dict()
    del metadata["languages"]
    metadata["source"] = metadata["filename"]
    documents.append(Document(page_content=element.text, metadata=metadata))

In [ ]:
len(documents)

731

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
qdrant_url = os.getenv("QDRANT_URL")
qdrant_api_key = os.getenv("QDRANT_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [ ]:
embeddings = FastEmbedEmbeddings()

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 78251.94it/s]


In [ ]:
# this will take some time, patience is the key :)
vectorstore = Qdrant.from_documents(documents=documents,
                                    embedding = embeddings,
                                    url = qdrant_url,
                                    collection_name="rag",
                                    api_key=qdrant_api_key)

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

## Creating RAG (Qdrant, Groq, LangChain, Llama3)

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains import chat_history_aware_retriever
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [ ]:
template = """You are an AI assistant for answering questions about the GPT4All paper and Quarterly Report Pursuant to Section 13 or 15(d) of the Securities Exchange Act of 1934 for the quarterly period ended March 31, 2022.
You are given the following extracted parts of a long document and a question. Provide a conversational answer.
If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
Question: {question}
=========
{context}
=========
Answer in Markdown:"""
prompt = PromptTemplate(template=template, input_variables=["question", "context"])

In [ ]:
llm = ChatGroq(temperature=0,model_name="llama3-8b-8192")

doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")
question_generator_chain = LLMChain(llm=llm, prompt=prompt)
qa_chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator_chain,
    combine_docs_chain=doc_chain,
)

In [ ]:
qa_chain.invoke({
    "question": "What was the net loss including non-controlling interests of Uber in 2021", #line 533
    "chat_history": []
})["answer"]

'The net loss including non-controlling interests of Uber in 2021 was $(122) million.\nSOURCES: uber_10q_march_2022.md'

In [ ]:
# hybrid search in action
filter_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1, "filter": {"source": "gpt4all.pdf"}}
)

In [ ]:
filter_chain = ConversationalRetrievalChain(
    retriever=filter_retriever,
    question_generator=question_generator_chain,
    combine_docs_chain=doc_chain,
)

In [ ]:
filter_chain.invoke({
    "question": "How was GPT4All-Snoozy developed ?",
    "chat_history": [],
    "filter": filter,
})["answer"]

"I'm happy to help!\n\nFINAL ANSWER: The president did not mention Michael Jackson.\nSOURCES:\n\nFINAL ANSWER: This Agreement is governed by English law.\nSOURCES: 28-pl\n\nFINAL ANSWER: The president did not mention Michael Jackson.\nSOURCES:\n\nFINAL ANSWER: GPT4All-Snoozy was developed using roughly the same procedure as the previous GPT4All models, but with a few key modifications.\nSOURCES: gpt4all.pdf"